In [7]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import random
import math
import sys
import copy
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore') 
from zipfile import ZipFile 
from collections import Counter
import cv2
from collections import deque



In [25]:
cap = cv2.VideoCapture(0)
# cap.set(3,500)
# cap.set(4,500)
IMAGE_SHAPE = (480,640)
color_index = 0
colors=[(255,0,0) ,(0,255,0) ,(0,0,255) ,(0,255,255)]
kernel = np.ones((5,5),np.uint8)
bpoints = [deque(maxlen=1024)]
gpoints = [deque(maxlen=1024)]
rpoints = [deque(maxlen=1024)]
ypoints = [deque(maxlen=1024)]
r_index = 0
b_index  =0
g_index = 0
y_index  =0

# Here is code for Canvas setup
paintWindow = np.zeros((480,640,3)) + 255
paintWindow = cv2.rectangle(paintWindow, (590,0), (639,50), (0,0,0), 2)
paintWindow = cv2.rectangle(paintWindow, (590,100), (639,150), colors[0], -1)
paintWindow = cv2.rectangle(paintWindow, (590,200), (639,250), colors[1], -1)
paintWindow = cv2.rectangle(paintWindow, (590,300), (639,350), colors[2], -1)
paintWindow = cv2.rectangle(paintWindow, (590,400), (639,450), colors[3], -1)
cv2.putText(paintWindow, "X", (595, 45), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255),7, cv2.LINE_AA)
 


cv2.imshow('paintWindow' ,paintWindow)


store=[]

while True:
    ret , frame= cap.read()
    frame =  cv2.flip(frame ,1)
    hsv = cv2.cvtColor(frame , cv2.COLOR_BGR2HSV)
   
    
    frame = cv2.rectangle(frame, (590,0), (639,50), (0,0,0), 2)
    frame = cv2.rectangle(frame, (590,100), (639,150), colors[0], -1)
    frame = cv2.rectangle(frame, (590,200), (639,250), colors[1], -1)
    frame = cv2.rectangle(frame, (590,300), (639,350), colors[2], -1)
    frame = cv2.rectangle(frame, (590,400), (639,450), colors[3], -1)
    cv2.putText(frame, "X", (595, 45), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255),7, cv2.LINE_AA)
    
    
    lower =np.array([0, 120, 70])
    upper = np.array([10, 255, 255])
    mask  = cv2.inRange(hsv , lower, upper)
    mask = cv2.erode(mask ,kernel , iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask =cv2.dilate(mask , kernel ,iterations=1)
    
    cnts , _   = cv2.findContours(mask.copy() ,cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_SIMPLE)
    center =None
    
    if len(cnts)>0:
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
#         print(x,y)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
        store.append(center) 
        
        if center[0]>590:
            if 0<=center[1]<=50:  #clear button
                bpoints =[deque(maxlen=512)]
                gpoints =[deque(maxlen=512)]
                rpoints =[deque(maxlen=512)]
                ypoints =[deque(maxlen=512)]
                r_index = 0
                b_index  =0
                g_index = 0
                y_index  =0
                paintWindow[50:,:,:]= 255
            elif 100<=center[1]<=150:
                color_index=0
            elif 200<=center[1]<=250:
                color_index=1   
            elif 300<=center[1]<=350:
                color_index=2    
            elif 400<=center[1]<=450:
                color_index=3    
        else:
            if color_index==0:
                bpoints[b_index].appendleft(center)
            if color_index==1:
                gpoints[g_index].appendleft(center)    
            if color_index==2:
                rpoints[r_index].appendleft(center)
            if color_index==3:
                ypoints[y_index].appendleft(center)    
    else:
        bpoints.append(deque(maxlen =512))
        b_index+=1
        gpoints.append(deque(maxlen =512))
        g_index+=1
        rpoints.append(deque(maxlen =512))
        r_index+=1
        ypoints.append(deque(maxlen =512))
        y_index+=1
        
    points=[bpoints ,gpoints,rpoints,ypoints]
    for i in range(0,len(points)):
        for j in range(0,len(points[i])):
            for k in range(1,len(points[i][j])):
                if points[i][j][k-1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame , points[i][j][k-1] ,points[i][j][k] ,colors[i] ,2)
                cv2.line(paintWindow , points[i][j][k-1]  ,points[i][j][k] ,colors[i] ,2)
    
    cv2.imshow('frame', frame)
    cv2.imshow('paintWindow', paintWindow)
    cv2.imshow('mask', mask)
         

    k = cv2.waitKey(5)
    if k==27:
        break
cap.release()
cv2.destroyAllWindows()


